In [ ]:
#import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg') # Must be before importing matplotlib.pyplot or pylab!
import matplotlib.pyplot as plt

import numpy as np
import sys
sys.path.insert(0,'/local2/home/tong/caffe-master/python')
import caffe

import os
from matplotlib import rc
rc('mathtext', default='regular')

from sklearn.metrics import confusion_matrix

caffe.set_mode_gpu()
caffe.set_device(0)


In [ ]:
#for training
solver = caffe.SGDSolver('/local2/home/tong/fashionRecommendation/models/fashionNet_3/fashion_solver_3.prototxt')
#CHANGE!!!
# solver.net.copy_from('/local2/home/tong/fashionRecommendation/models/VGG_M_2048/VGG_CNN_M_2048.caffemodel')
#CHANGE!!!
# solver.test_nets[0].copy_from('/local2/home/tong/fashionRecommendation/models/VGG_M_2048/VGG_CNN_M_2048.caffemodel')
# solver.net.copy_from('/local2/home/tong/fashionRecommendation/models/fashionNet_2/training_record/fashion_params_2_91448.caffemodel')
# solver.test_nets[0].copy_from('/local2/home/tong/fashionRecommendation/models/fashionNet_2/training_record/fashion_params_2_91448.caffemodel')


In [ ]:
def test_avg(test_iter,iters):
    avg_accu = 0
    avg_loss = 0
    y_l_true = []
    y_l_pred = []
    y_r_true = []
    y_r_pred = []
    dd_dislike = 0
    dd_count = 0
    dl_dislike = 0
    dl_count = 0
    ld_like = 0
    ld_count = 0
    ll_like = 0
    ll_count = 0
    nr_tuples_pos = []
    count_posi = 0
    nr_tuples_neg = []
    count_nega = 0
    scores_pos = []
    scores_neg = []
    for i in range(0,test_iter):
        if (i%50==0 or (i==(test_iter-1))):
            print("{}/{} @ {}\n".format(i+1,test_iter,iters))
        solver.test_nets[0].forward()
        avg_accu = avg_accu + solver.test_nets[0].blobs['accuracy'].data
        avg_loss = avg_loss + solver.test_nets[0].blobs['rank_Loss'].data
        for j in range(0,len(solver.test_nets[0].blobs['label_top'].data)):
            y_l_pred.append(0 if solver.test_nets[0].blobs['metric_fc3'].data[j][0]>solver.test_nets[0].blobs['metric_fc3'].data[j][1] else 1)
            y_l_true.append(1)
            # count posi tuple number
            count_posi = count_posi + 1
            # record pre_score for this posi outfit
            scores_pos.append(solver.test_nets[0].blobs['metric_fc3_softmax'].data[j][1])
            y_r_pred.append(0 if solver.test_nets[0].blobs['metric_fc3_n'].data[j][0]>solver.test_nets[0].blobs['metric_fc3_n'].data[j][1] else 1)
            y_r_true.append(0)
            # count neg tuple number
            count_nega = count_nega + 1
            # record pre_score for this nega outfit
            scores_neg.append(solver.test_nets[0].blobs['metric_fc3_softmax_n'].data[j][1])
            #dd
            if (solver.test_nets[0].blobs['metric_fc3_n'].data[j][0]>solver.test_nets[0].blobs['metric_fc3_n'].data[j][1]):
                dd_count = dd_count + 1
                dd_dislike = dd_dislike + solver.test_nets[0].blobs['metric_fc3_softmax_n'].data[j][0]
            #dl
            else:
                dl_count = dl_count + 1
                dl_dislike = dl_dislike + solver.test_nets[0].blobs['metric_fc3_softmax_n'].data[j][0]
            #ld
            if (solver.test_nets[0].blobs['metric_fc3'].data[j][0]>solver.test_nets[0].blobs['metric_fc3'].data[j][1]):
                ld_count = ld_count + 1
                ld_like = ld_like + solver.test_nets[0].blobs['metric_fc3_softmax'].data[j][1]
            #ll
            else:
                ll_count = ll_count + 1
                ll_like = ll_like + solver.test_nets[0].blobs['metric_fc3_softmax'].data[j][1]
    cMat_l = confusion_matrix(y_l_true, y_l_pred)
    if(len(cMat_l)==1):
        cMat_l = [[0,0],[0,cMat_l[0][0]]]
    cMat_r = confusion_matrix(y_r_true, y_r_pred)
    if(len(cMat_r)==1):
        cMat_r = [[cMat_r[0][0],0],[0,0]]
    nr_tuples_pos.append(count_posi)
    nr_tuples_neg.append(count_nega)
    
    return (avg_accu/test_iter), (avg_loss/test_iter),cMat_l,cMat_r, \
           (0 if dd_count==0 else dd_dislike/dd_count), (0 if dl_count==0 else dl_dislike/dl_count), ((dd_dislike+dl_dislike)/(dd_count+dl_count)), \
           (0 if ld_count==0 else ld_like/ld_count), (0 if ll_count==0 else ll_like/ll_count), ((ld_like+ll_like)/(ld_count+ll_count)), \
           scores_pos, scores_neg, nr_tuples_pos, nr_tuples_neg


In [ ]:
def get_ndcg(scores_pos, scores_neg, nr_tuples_pos, nr_tuples_neg,
			 fn_out='', tuples_pos=None, tuples_neg=None, nr_return=0):
	"""
	- nr_tuples_pos[ui]: the number of positive outfits for user ui
	- nr_tuples_neg[ui]: the number of negative(neutral) outfits for user ui
	
	- scores_pos: scores for positive outfits. 
				  scores_pos[0:nr_tuples_pos[0]] are the scores for the first user.
				  scores_pos[nr_tuples_pos[0]:nr_tuples_pos[0]+nr_tuples_pos[1]] are the scores for the second user.
				
	- scores_neg: scores for negative outfits. 
				  scores_neg[0:nr_tuples_neg[0]] are the scores for the first user.
				  scores_neg[nr_tuples_neg[0]:nr_tuples_neg[0]+nr_tuples_neg[1]] are the scores for the second user.
	"""
	if fn_out != '':
            fid_out = open(fn_out, 'w')

	m = 10 # pre-determined ndcg size
	ndcg_ct = np.zeros(m)
	ndcg_at = np.zeros(m)
	mean_ndcg = 0
	s_ind_pos = 0 # update posi outfit index for each user
	s_ind_neg = 0 # update neutral outfit index for each user
	nr_users = len(nr_tuples_pos) # user number
	for ui in range(nr_users): # for each user
		count_q = nr_tuples_pos[ui] + nr_tuples_neg[ui] # total outfits (both posi & neutral) number of this user
		label = np.zeros(nr_tuples_pos[ui]+nr_tuples_neg[ui]) # labels for all outfits of this user, 1 for posi & 0 for neutral
		label[:nr_tuples_pos[ui]] = 1
		target = np.empty(nr_tuples_pos[ui]+nr_tuples_neg[ui]) # scores for all outfits (both posi & neutral)
		target[:nr_tuples_pos[ui]] = scores_pos[s_ind_pos:s_ind_pos+nr_tuples_pos[ui]] # scores for posi outfits
		target[nr_tuples_pos[ui]:] = scores_neg[s_ind_neg:s_ind_neg+nr_tuples_neg[ui]] # scores for neutral outfits
		if fn_out != '':
			tuples = np.hstack((tuples_pos[:,s_ind_pos:s_ind_pos+nr_tuples_pos[ui]],
								tuples_neg[:,s_ind_neg:s_ind_neg+nr_tuples_neg[ui]]))
		s_ind_pos += nr_tuples_pos[ui]
		s_ind_neg += nr_tuples_neg[ui]

		ndcg_size = min(m, count_q) # actual ndcg size
		ideal_dcg = np.empty(count_q) # for computing ideal ndcg value
		dcg = np.empty(count_q) # for computing dcg (without normalization by N_m yet)
		ndcg = 0
		order = np.argsort(-label) # sort label in descending order, returns the sequential indices of label 
		ideal_dcg[0] = pow(2.0, label[order[0]]) - 1 # compute ideal_ndcg@m (m=1)
		for i in range(1, count_q): # compute ideal_ndcg@m (m=2,...,M), M is total outfits number of this user
			ideal_dcg[i] = ideal_dcg[i-1]+(pow(2.0, label[order[i]])
										   - 1)*np.log(2.0)/np.log(i+1.0)
		order = np.argsort(-target) # sort scores for all outfits in descending order, returns the indices
		dcg[0] = pow(2.0, label[order[0]]) - 1 # compute dcg@m (m=1)
		for i in range(1, count_q): # compute dcg@m (m=2,...,M), M is total outfits number of this user
			dcg[i] = dcg[i-1]+(pow(2.0, label[order[i]])
							   - 1)*np.log(2.0)/np.log(i+1.0)
		if ideal_dcg[0] > 0: # at least there should be one posi outfit for this user, or else somehting is wrong here
			for i in range(count_q): # for each @m (m=1,...,M)
				ndcg += dcg[i] / ideal_dcg[i] # add up all ndcg@m (m=1,..,M) for this user
			for i in range(ndcg_size): # compute top 10 ndcg for all users
				ndcg_ct[i] += 1 # record outfits num at each place, among top 10
				ndcg_at[i] += dcg[i] / ideal_dcg[i] # add up ndcg value at each place, among top 10
		else: # if we only have no posi outfit for this user
			ndcg = 0 # ndcg is 0 for this user, because there is not point of ranking anymore for him/here
		m_ndcg = ndcg / count_q # mean ndcg for this user
		mean_ndcg += m_ndcg # add up mean ndcg for all users

		if fn_out != '':
			fid_out.write('%f\n' % m_ndcg)
			n_out = min(count_q, nr_return)
			for i in range(n_out):
				fid_out.write('%d ' % label[order[i]])
				for jj in range(tuples.shape[0]):
					fid_out.write('%d ' % tuples[jj, order[i]])
				fid_out.write('\n')

	mean_ndcg /= nr_users # mean ndcg for all users as a whole
	for i in range(m): # top 10 mean ndcg for all users as a whole
		ndcg_at[i] /= ndcg_ct[i]

	if fn_out != '':
		fid_out.close()

	return (mean_ndcg, ndcg_at)

In [ ]:
# test_iter = 450
# test_accu,test_loss,cMat_l,cMat_r=test_avg(test_iter)
# print("test_accu: {}\n".format(test_accu))
# print("test_loss: {}\n".format(test_loss))
# print("cMat_l:\n")
# print(cMat_l)
# print("\ncMat_r:\n")
# print(cMat_r)


In [ ]:
recordDir = '/local2/home/tong/fashionRecommendation/models/fashionNet_2/training_record/'
test_interval = 920 # 1/10 train epoch
test_iter = 90 # 1/5 test epoch
start_iter = 0
end_iter = 137706 #7+8 epoch
for i in range(start_iter,end_iter+1):
    if (i%test_interval==0 or i==end_iter):
        #validation & conf_matrix
        solver.test_nets[0].copy_from(recordDir+'fashion_params_2_'+str(i)+'.caffemodel')
        test_accu,test_loss,cMat_l,cMat_r,dd_dislike,dl_dislike,d_dislike,ld_like,ll_like,l_like,scores_pos, scores_neg, nr_tuples_pos, nr_tuples_neg=test_avg(test_iter,i)
        mean_ndcg, ndcg_at = get_ndcg(scores_pos, scores_neg, nr_tuples_pos, nr_tuples_neg)
        test_rank_accu = open(recordDir + 'test_rank_accu.txt','a')
        test_rank_loss = open(recordDir + 'test_rank_loss.txt','a')
        test_rank_accu.write(str(i)+' '+str(test_accu)+'\r\n')
        test_rank_loss.write(str(i)+' '+str(test_loss)+'\r\n')
        test_rank_accu.close()
        test_rank_loss.close()
        te_l_conf = open(recordDir + 'conf_l_matrix.txt','a')
        te_l_conf.write(str(i)+' '+str(cMat_l[0][0])+' '+str(cMat_l[0][1])+' '+str(cMat_l[1][0])+' '+str(cMat_l[1][1])+'\r\n')
        te_l_conf.close()
        te_r_conf = open(recordDir + 'conf_r_matrix.txt','a')
        te_r_conf.write(str(i)+' '+str(cMat_r[0][0])+' '+str(cMat_r[0][1])+' '+str(cMat_r[1][0])+' '+str(cMat_r[1][1])+'\r\n')
        te_r_conf.close()
        dd_f = open(recordDir + 't2.1_rank/dd_dislike.txt','a')
        dd_f.write(str(i)+' '+str(dd_dislike)+'\r\n')
        dd_f.close()
        dl_f = open(recordDir + 't2.1_rank/dl_dislike.txt','a')
        dl_f.write(str(i)+' '+str(dl_dislike)+'\r\n')
        dl_f.close()
        d_f = open(recordDir + 't2.1_rank/d_dislike.txt','a')
        d_f.write(str(i)+' '+str(d_dislike)+'\r\n')
        d_f.close()
        ld_f = open(recordDir + 't2.1_rank/ld_like.txt','a')
        ld_f.write(str(i)+' '+str(ld_like)+'\r\n')
        ld_f.close()
        ll_f = open(recordDir + 't2.1_rank/ll_like.txt','a')
        ll_f.write(str(i)+' '+str(ll_like)+'\r\n')
        ll_f.close()
        l_f = open(recordDir + 't2.1_rank/l_like.txt','a')
        l_f.write(str(i)+' '+str(l_like)+'\r\n')
        l_f.close()
        # record mean_ndcg & ndcg_at
        ndcg_f = open(recordDir+'t2.1_rank/ndcg.txt','a')
        ndcg_f.write(str(i)+' '+str(mean_ndcg)+'\r\n')
        for n in range(0,len(ndcg_at)):
            ndcg_f.write(str(i)+' '+str(ndcg_at[n])+'\r\n')
        ndcg_f.close()
        print("\nIters done:{}, VAL_accu={}, VAL_loss={}.\n".format(i,test_accu,test_loss))
        #visualization
        test_accu_whole = []
        test_loss_whole = []
        test_accu_whole = open(recordDir+'test_rank_accu.txt').readlines()
        test_loss_whole = open(recordDir+'test_rank_loss.txt').readlines()
        test_iter_idx = []
        test_loss = []
        test_accuracy = []
        for i in range(0, len(test_accu_whole)):
            #test_iter_idx
            test_iter_idx.append(int(test_accu_whole[i].strip('\r\n').split(' ')[0]))
            #test_loss
            test_loss.append(float(test_loss_whole[i].strip('\r\n').split(' ')[1]))
            #test_accuracy
            test_accuracy.append(float(test_accu_whole[i].strip('\r\n').split(' ')[1]))
        fig = plt.figure()
        ax_left = fig.add_subplot(111)
        ax_left.plot(test_iter_idx, test_loss, '--gp', label = 'Avg_V_Loss')
        ax_right = ax_left.twinx()
        ax_right.plot(test_iter_idx, test_accuracy, '-yp', label = 'Avg_V_Accuracy')
        # ask matplotlib for the plotted objects and their labels
        lines_left, labels_left = ax_left.get_legend_handles_labels()
        lines_right, labels_right = ax_right.get_legend_handles_labels()
        ax_right.legend(lines_left + lines_right, labels_left + labels_right, loc=0)
        ax_left.grid()
        ax_left.set_xlabel("Training Iterations Done(n)")
        ax_left.set_ylabel("Loss")
        ax_right.set_ylabel("Accuracy")
        ax_right.set_title("HVA@({:.3f},{}), LVL@({:.3f},{})".format(max(test_accuracy),test_iter_idx[test_accuracy.index(max(test_accuracy))],min(test_loss),test_iter_idx[test_loss.index(min(test_loss))]))
        plt.savefig(recordDir+'rank_record.png', bbox_inches='tight')
        plt.close('all')
        conf_l_matrix_whole = []
        conf_r_matrix_whole = []
        conf_l_matrix_whole = open(recordDir+'conf_l_matrix.txt').readlines()
        conf_r_matrix_whole = open(recordDir+'conf_r_matrix.txt').readlines()
        conf_matrix = []
        for i in range(0,len(conf_l_matrix_whole)):
            conf_matrix.append([int(conf_r_matrix_whole[i].split(' ')[1]),int(conf_r_matrix_whole[i].split(' ')[2]), \
                                int(conf_l_matrix_whole[i].split(' ')[3]),int(conf_l_matrix_whole[i].split(' ')[4])])
        fig = plt.figure()
        ax = fig.add_subplot(111)
        im = ax.imshow(np.array(conf_matrix).T, interpolation='nearest', cmap=plt.cm.jet)
        fig.colorbar(im, ax=ax)
        plt.xlabel('Caffemodel Index(n)')
        plt.ylabel('like(like,dislike), dislike(like,dislike)')
        plt.title('Confusion Matrix@(dislike,like)')
        ax.set_aspect(25)
        fig.set_size_inches(20.5, 12.5)
        fig.savefig(recordDir+'t2.1_rank/cMat.png',bbox_inches='tight')
        plt.close('all')
        dd_dislike_whole = []
        dl_dislike_whole = []
        d_dislike_whole = []
        ld_like_whole = []
        ll_like_whole = []
        l_like_whole = []
        dd_dislike_whole = open(recordDir+'t2.1_rank/dd_dislike.txt').readlines()
        dl_dislike_whole = open(recordDir+'t2.1_rank/dl_dislike.txt').readlines()
        d_dislike_whole = open(recordDir+'t2.1_rank/d_dislike.txt').readlines()
        ld_like_whole = open(recordDir+'t2.1_rank/ld_like.txt').readlines()
        ll_like_whole = open(recordDir+'t2.1_rank/ll_like.txt').readlines()
        l_like_whole = open(recordDir+'t2.1_rank/l_like.txt').readlines()        
        iters_idx = []
        dd_dislike = []
        dd_like = []
        dl_dislike = []
        dl_like = []
        d_dislike = []
        d_like = []
        ld_like = []
        ld_dislike = []
        ll_like = []
        ll_dislike = []
        l_like = []
        l_dislike = []
        for i in range(0, len(dd_dislike_whole)):
            iters_idx.append(int(dd_dislike_whole[i].strip('\r\n').split(' ')[0]))
            dd_dislike.append(float(dd_dislike_whole[i].strip('\r\n').split(' ')[1]))
            dd_like.append(1-dd_dislike[i])
            dl_dislike.append(float(dl_dislike_whole[i].strip('\r\n').split(' ')[1]))
            dl_like.append(1-dl_dislike[i])
            d_dislike.append(float(d_dislike_whole[i].strip('\r\n').split(' ')[1]))
            d_like.append(1-d_dislike[i])
            ld_like.append(float(ld_like_whole[i].strip('\r\n').split(' ')[1]))
            ld_dislike.append(1-ld_like[i])
            ll_like.append(float(ll_like_whole[i].strip('\r\n').split(' ')[1]))
            ll_dislike.append(1-ll_like[i])
            l_like.append(float(l_like_whole[i].strip('\r\n').split(' ')[1]))
            l_dislike.append(1-l_like[i])
        #dd
        fig = plt.figure()
        ax_left = fig.add_subplot(111)
        ax_left.plot(iters_idx, dd_dislike, '--rp', label = 'dd_dislike')
        ax_left.plot(iters_idx, dd_like, '--gp', label = 'dd_like')
        lines_left, labels_left = ax_left.get_legend_handles_labels()   
        ax_left.legend(lines_left, labels_left, loc=0)
        ax_left.grid()
        ax_left.set_xlabel("Training Iterations Done(n)")
        ax_left.set_ylabel("Softmax Probability")
        ax_left.set_title("Dislike to Dislike [correct]")
        plt.savefig(recordDir+'t2.1_rank/dd_record.png', bbox_inches='tight')
        plt.close('all')
        #dl
        fig = plt.figure()
        ax_left = fig.add_subplot(111)
        ax_left.plot(iters_idx, dl_dislike, '--rp', label = 'dl_dislike')
        ax_left.plot(iters_idx, dl_like, '--gp', label = 'dl_like')
        lines_left, labels_left = ax_left.get_legend_handles_labels()   
        ax_left.legend(lines_left, labels_left, loc=0)
        ax_left.grid()
        ax_left.set_xlabel("Training Iterations Done(n)")
        ax_left.set_ylabel("Softmax Probability")
        ax_left.set_title("Dislike to Like [wrong]")
        plt.savefig(recordDir+'t2.1_rank/dl_record.png', bbox_inches='tight')        
        plt.close('all')
        #d
        fig = plt.figure()
        ax_left = fig.add_subplot(111)
        ax_left.plot(iters_idx, d_dislike, '--rp', label = 'd_dislike')
        ax_left.plot(iters_idx, d_like, '--gp', label = 'd_like')
        lines_left, labels_left = ax_left.get_legend_handles_labels()   
        ax_left.legend(lines_left, labels_left, loc=0)
        ax_left.grid()
        ax_left.set_xlabel("Training Iterations Done(n)")
        ax_left.set_ylabel("Softmax Probability")
        ax_left.set_title("Dislike to Both")
        plt.savefig(recordDir+'t2.1_rank/d_record.png', bbox_inches='tight')
        plt.close('all')
        #ld
        fig = plt.figure()
        ax_left = fig.add_subplot(111)
        ax_left.plot(iters_idx, ld_dislike, '--rp', label = 'ld_dislike')
        ax_left.plot(iters_idx, ld_like, '--gp', label = 'ld_like')
        lines_left, labels_left = ax_left.get_legend_handles_labels()   
        ax_left.legend(lines_left, labels_left, loc=0)
        ax_left.grid()
        ax_left.set_xlabel("Training Iterations Done(n)")
        ax_left.set_ylabel("Softmax Probability")
        ax_left.set_title("Like to Dislike [wrong]")
        plt.savefig(recordDir+'t2.1_rank/ld_record.png', bbox_inches='tight')
        plt.close('all')
        #ll
        fig = plt.figure()
        ax_left = fig.add_subplot(111)
        ax_left.plot(iters_idx, ll_dislike, '--rp', label = 'll_dislike')
        ax_left.plot(iters_idx, ll_like, '--gp', label = 'll_like')
        lines_left, labels_left = ax_left.get_legend_handles_labels()   
        ax_left.legend(lines_left, labels_left, loc=0)
        ax_left.grid()
        ax_left.set_xlabel("Training Iterations Done(n)")
        ax_left.set_ylabel("Softmax Probability")
        ax_left.set_title("Like to Like [correct]")
        plt.savefig(recordDir+'t2.1_rank/ll_record.png', bbox_inches='tight')
        plt.close('all')
        #l
        fig = plt.figure()
        ax_left = fig.add_subplot(111)
        ax_left.plot(iters_idx, l_dislike, '--rp', label = 'l_dislike')
        ax_left.plot(iters_idx, l_like, '--gp', label = 'l_like')
        lines_left, labels_left = ax_left.get_legend_handles_labels()   
        ax_left.legend(lines_left, labels_left, loc=0)
        ax_left.grid()
        ax_left.set_xlabel("Training Iterations Done(n)")
        ax_left.set_ylabel("Softmax Probability")
        ax_left.set_title("Like to Both")
        plt.savefig(recordDir+'t2.1_rank/l_record.png', bbox_inches='tight')
        plt.close('all')
        # mean_ndcg & ndcg_at
        ndcg_whole = []
        ndcg_whole = open(recordDir+'t2.1_rank/ndcg.txt').readlines()
        ndcg_idx = []
        mean_ndcg = []
        ndcg_1 = []
        ndcg_2 = []
        ndcg_3 = []
        ndcg_4 = []
        ndcg_5 = []
        ndcg_6 = []
        ndcg_7 = []
        ndcg_8 = []
        ndcg_9 = []
        ndcg_10 = []
        for i in range(0,len(ndcg_whole)):
            if (i % 11 == 0):
                ndcg_idx.append(int(ndcg_whole[i+0].strip('\r\n').split(' ')[0]))
                mean_ndcg.append(float(ndcg_whole[i+0].strip('\r\n').split(' ')[1]))
                ndcg_1.append(float(ndcg_whole[i+1].strip('\r\n').split(' ')[1]))
                ndcg_2.append(float(ndcg_whole[i+2].strip('\r\n').split(' ')[1]))
                ndcg_3.append(float(ndcg_whole[i+3].strip('\r\n').split(' ')[1]))
                ndcg_4.append(float(ndcg_whole[i+4].strip('\r\n').split(' ')[1]))
                ndcg_5.append(float(ndcg_whole[i+5].strip('\r\n').split(' ')[1]))
                ndcg_6.append(float(ndcg_whole[i+6].strip('\r\n').split(' ')[1]))
                ndcg_7.append(float(ndcg_whole[i+7].strip('\r\n').split(' ')[1]))
                ndcg_8.append(float(ndcg_whole[i+8].strip('\r\n').split(' ')[1]))
                ndcg_9.append(float(ndcg_whole[i+9].strip('\r\n').split(' ')[1]))
                ndcg_10.append(float(ndcg_whole[i+10].strip('\r\n').split(' ')[1]))
        fig = plt.figure()
        ax_left = fig.add_subplot(111)
        ax_left.plot(ndcg_idx, mean_ndcg, '--gp', label = 'mean_ndcg')
        ax_left.plot(ndcg_idx, ndcg_1, '--r', label = 'ndcg_1')
        ax_left.plot(ndcg_idx, ndcg_2, '--g', label = 'ndcg_2')
        ax_left.plot(ndcg_idx, ndcg_3, '--b', label = 'ndcg_3')
        ax_left.plot(ndcg_idx, ndcg_4, '--c', label = 'ndcg_4')
        ax_left.plot(ndcg_idx, ndcg_5, '--m', label = 'ndcg_5')
        ax_left.plot(ndcg_idx, ndcg_6, '--k', label = 'ndcg_6')
        ax_left.plot(ndcg_idx, ndcg_7, '-.y', label = 'ndcg_7')
        ax_left.plot(ndcg_idx, ndcg_8, '-.k', label = 'ndcg_8')
        ax_left.plot(ndcg_idx, ndcg_9, '-.b', label = 'ndcg_9')
        ax_left.plot(ndcg_idx, ndcg_10, '-.m', label = 'ndcg_10')
        lines_left, labels_left = ax_left.get_legend_handles_labels()   
        ax_left.legend(lines_left, labels_left, loc=0)
        ax_left.grid()
        ax_left.set_xlabel("Training Iterations Done(n)")
        ax_left.set_ylabel("Mean NDCGs")
        ax_left.set_title("Mean NDCG of Different Lengths")
        plt.savefig(recordDir+'t2.1_rank/ndcg.png', bbox_inches='tight')
        plt.close('all')
        

In [ ]:
import matplotlib
matplotlib.use('Agg') # Must be before importing matplotlib.pyplot or pylab!
import matplotlib.pyplot as plt
import numpy as np

recordDir = '/local2/home/tong/fashionRecommendation/models/fashionNet_2/training_record/'

conf_l_matrix_whole = []
conf_r_matrix_whole = []
conf_l_matrix_whole = open(recordDir+'conf_l_matrix.txt').readlines()
conf_r_matrix_whole = open(recordDir+'conf_r_matrix.txt').readlines()

# conf_l_matrix_whole = []
# conf_r_matrix_whole = []
# conf_l_matrix_whole = open('conf_l_matrix.txt').readlines()
# conf_r_matrix_whole = open('conf_r_matrix.txt').readlines()
conf_matrix = []
for i in range(0,len(conf_l_matrix_whole)):
    conf_matrix.append([int(conf_r_matrix_whole[i].split(' ')[1]),int(conf_r_matrix_whole[i].split(' ')[2]), \
                        int(conf_l_matrix_whole[i].split(' ')[3]),int(conf_l_matrix_whole[i].split(' ')[4])])
fig = plt.figure()
ax = fig.add_subplot(111)
im = ax.imshow(np.array(conf_matrix).T, interpolation='nearest', cmap=plt.cm.jet)
fig.colorbar(im, ax=ax)
plt.xlabel('Caffemodel Index(n)')
plt.ylabel('like(like,dislike), dislike(like,dislike)')
plt.title('Confusion Matrix@(dislike,like)')
ax.set_aspect(25)
fig.set_size_inches(20.5, 12.5)
fig.savefig(recordDir+'cMat.png',bbox_inches='tight')
# fig.savefig('cMat.png',bbox_inches='tight')


In [ ]:
# save 2 caffemodels for user_specific fine tuning
recordDir = '/local2/home/tong/fashionRecommendation/models/fashionNet_2/training_record/'

# 1) [90000,110000]
# caffemodel_idx:
ndcg_whole = []
ndcg_whole = open(recordDir+'t2.1_rank/ndcg.txt').readlines()
ndcg_idx = []
mean_ndcg = []
ndcg_10 = []
for i in range(0,len(ndcg_whole)):
    if (i % 11 == 0):
        ndcg_idx_temp = int(ndcg_whole[i+0].strip('\r\n').split(' ')[0])
        if (ndcg_idx_temp>=90000 and ndcg_idx_temp<=110000):
            ndcg_idx.append(ndcg_idx_temp)
            mean_ndcg.append(float(ndcg_whole[i+0].strip('\r\n').split(' ')[1]))
            ndcg_10.append(float(ndcg_whole[i+10].strip('\r\n').split(' ')[1]))
print("H_mean_ndcg@({:.3f},{}), C_ndcg_10@({:.3f})".format(max(mean_ndcg),ndcg_idx[mean_ndcg.index(max(mean_ndcg))],ndcg_10[mean_ndcg.index(max(mean_ndcg))]))

test_accu_whole = []
test_loss_whole = []
test_accu_whole = open(recordDir+'test_rank_accu.txt').readlines()
test_loss_whole = open(recordDir+'test_rank_loss.txt').readlines()
test_iter_idx = []
test_loss = []
test_accuracy = []
for i in range(0, len(test_accu_whole)):
    #test_iter_idx
    test_iter_idx.append(int(test_accu_whole[i].strip('\r\n').split(' ')[0]))
    #test_loss
    test_loss.append(float(test_loss_whole[i].strip('\r\n').split(' ')[1]))
    #test_accuracy
    test_accuracy.append(float(test_accu_whole[i].strip('\r\n').split(' ')[1]))
test_idx_temp = test_iter_idx.index(ndcg_idx[mean_ndcg.index(max(mean_ndcg))])
print("test_accu: {}, test_loss: {}".format(test_accuracy[test_idx_temp], test_loss[test_idx_temp]))

# 2) [110000,137706］
# caffemodel_idx:
ndcg_idx = []
mean_ndcg = []
ndcg_10 = []
for i in range(0,len(ndcg_whole)):
    if (i % 11 == 0):
        ndcg_idx_temp = int(ndcg_whole[i+0].strip('\r\n').split(' ')[0])
        if (ndcg_idx_temp>=110000 and ndcg_idx_temp<=137706):
            ndcg_idx.append(ndcg_idx_temp)
            mean_ndcg.append(float(ndcg_whole[i+0].strip('\r\n').split(' ')[1]))
            ndcg_10.append(float(ndcg_whole[i+10].strip('\r\n').split(' ')[1]))
print("H_mean_ndcg@({:.3f},{}), C_ndcg_10@({:.3f})".format(max(mean_ndcg),ndcg_idx[mean_ndcg.index(max(mean_ndcg))],ndcg_10[mean_ndcg.index(max(mean_ndcg))]))

test_accu_whole = []
test_loss_whole = []
test_accu_whole = open(recordDir+'test_rank_accu.txt').readlines()
test_loss_whole = open(recordDir+'test_rank_loss.txt').readlines()
test_iter_idx = []
test_loss = []
test_accuracy = []
for i in range(0, len(test_accu_whole)):
    #test_iter_idx
    test_iter_idx.append(int(test_accu_whole[i].strip('\r\n').split(' ')[0]))
    #test_loss
    test_loss.append(float(test_loss_whole[i].strip('\r\n').split(' ')[1]))
    #test_accuracy
    test_accuracy.append(float(test_accu_whole[i].strip('\r\n').split(' ')[1]))  
test_idx_temp = test_iter_idx.index(ndcg_idx[mean_ndcg.index(max(mean_ndcg))])
print("test_accu: {}, test_loss: {}".format(test_accuracy[test_idx_temp], test_loss[test_idx_temp]))
